In [1]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import string
#import tensorflow as tf

In [2]:
pd.set_option("display.max_row",None)
pd.set_option("display.max_column",None)
df = pd.read_csv("tweeter.csv")


In [3]:
df.head()

,label,id,date,QUERY,_rev,tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,_TheSpecialOne_,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,_TheSpecialOne_,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,_TheSpecialOne_,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,_TheSpecialOne_,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,_TheSpecialOne_,@Kwesidei not the whole crew


In [4]:
df = df[['label','id','tweet']]

In [5]:
df.head()

,label,id,tweet
0,0,1467810672,is upset that he can't update his Facebook by ...
1,0,1467810917,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,my whole body feels itchy and like its on fire
3,0,1467811193,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,@Kwesidei not the whole crew


# cleaning tweets

In [6]:
def clean(text):
    text = re.sub(r'@[A-Za-z0-9]+','',text)  #mentions
    text = re.sub(r'#+','',text)   #symbol #
    text = re.sub(r'https?:\/\/\S+','',text)   #hyperlinks
    text = re.sub(r'RT[\s]+','',text)  #RT
    text = text.lower()
    text = text.translate(str.maketrans("","",string.punctuation))
    
    return text

In [7]:
df['tweet'][0]

"is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"

In [8]:
df['label'].unique()

array([0, 4], dtype=int64)

In [9]:
df['label']=df['label'].replace(4,1)

In [10]:
df['label'].unique()

array([0, 1], dtype=int64)

In [11]:
df2=df[:10000]

In [12]:
df2['label'].unique()

array([0], dtype=int64)

In [13]:
df2 = df2.append(df[890000:900000])

In [14]:
df2.reset_index(drop=True,inplace=True)

In [15]:
df['clean_tweet']=''

In [16]:
df2.shape

(20000, 3)

In [17]:
df2.tail()

,label,id,tweet
19995,1,1693956096,@altitis and to you!
19996,1,1693956099,Okie doke!! Time for me to escape for the Nort...
19997,1,1693956134,"finished the lessons, hooray!"
19998,1,1693956160,Some ppl are just fucking KP0. Cb ! Stop askin...
19999,1,1693956175,Thanks for pointing out the crucial problems @...


In [18]:
df2['label'].unique()

array([0, 1], dtype=int64)

In [19]:
df2['clean_tweet']=''

In [20]:
df2.head()

,label,id,tweet,clean_tweet
0,0,1467810672,is upset that he can't update his Facebook by ...,
1,0,1467810917,@Kenichan I dived many times for the ball. Man...,
2,0,1467811184,my whole body feels itchy and like its on fire,
3,0,1467811193,"@nationwideclass no, it's not behaving at all....",
4,0,1467811372,@Kwesidei not the whole crew,


In [21]:
n=0
for i in df2['tweet']:
    df2['clean_tweet'][n] = clean(i)
    #print(n)
    n=n+1

<ipython-input-21-2092dde147f3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['clean_tweet'][n] = clean(i)


In [22]:
df2.head()

,label,id,tweet,clean_tweet
0,0,1467810672,is upset that he can't update his Facebook by ...,is upset that he cant update his facebook by t...
1,0,1467810917,@Kenichan I dived many times for the ball. Man...,i dived many times for the ball managed to sa...
2,0,1467811184,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
3,0,1467811193,"@nationwideclass no, it's not behaving at all....",no its not behaving at all im mad why am i he...
4,0,1467811372,@Kwesidei not the whole crew,not the whole crew


In [23]:
df2['clean_tweet'] = df2['clean_tweet'].apply(lambda x: " ".join([w for w in x.split() if len(w)>3]))
df2.tail()

,label,id,tweet,clean_tweet
19995,1,1693956096,@altitis and to you!,
19996,1,1693956099,Okie doke!! Time for me to escape for the Nort...,okie doke time escape north while massas back ...
19997,1,1693956134,"finished the lessons, hooray!",finished lessons hooray
19998,1,1693956160,Some ppl are just fucking KP0. Cb ! Stop askin...,some just fucking stop asking love boyfriend t...
19999,1,1693956175,Thanks for pointing out the crucial problems @...,thanks pointing crucial problems both them hav...


In [24]:
token = df2['clean_tweet'].apply(lambda x: x.split())

In [25]:
token.head()

0    [upset, that, cant, update, facebook, texting,...
1    [dived, many, times, ball, managed, save, rest...
2              [whole, body, feels, itchy, like, fire]
3         [behaving, here, because, cant, over, there]
4                                        [whole, crew]
Name: clean_tweet, dtype: object

In [26]:
ps = PorterStemmer()

In [27]:
ps.stem("drags")

'drag'

In [28]:
token = token.apply(lambda x: [ps.stem(word) for word in x])

In [29]:
token

0        [upset, that, cant, updat, facebook, text, mig...
1        [dive, mani, time, ball, manag, save, rest, bo...
2                   [whole, bodi, feel, itchi, like, fire]
3                 [behav, here, becaus, cant, over, there]
4                                            [whole, crew]
5                                                   [need]
6               [long, time, rain, onli, fine, thank, how]
7                                [nope, they, didnt, have]
8                                                  [muera]
9                       [spring, break, plain, citi, snow]
10                                    [just, repierc, ear]
11        [couldnt, bear, watch, thought, loss, embarrass]
12                    [count, either, never, talk, anymor]
13       [wouldv, been, first, didnt, have, realli, tho...
14                      [wish, watch, with, miss, premier]
15       [holli, death, scene, will, hurt, sever, watch...
16                                      [about, file, ta

In [30]:
for i in range(len(token)):
               token[i] = " ".join(token[i])
               print(token[i])
df2['clean_tweet'] = token

upset that cant updat facebook text might result school today also blah
dive mani time ball manag save rest bound
whole bodi feel itchi like fire
behav here becaus cant over there
whole crew
need
long time rain onli fine thank how
nope they didnt have
muera
spring break plain citi snow
just repierc ear
couldnt bear watch thought loss embarrass
count either never talk anymor
wouldv been first didnt have realli though snyder just doucheclown
wish watch with miss premier
holli death scene will hurt sever watch film director
about file tax
alway want rent love soundtrack
dear were drink forgotten tabl drink
most didnt much done
friend call ask meet with valley todaybut time sigh
barista bake cake ate
thi week go hope
blagh class tomorrow
hate when have call wake peopl
just go myself sleep after watch marley
misslilli
ooooh that lesli wont again lesli wont again
almost lover except thi track get depress everi time
some1 hack account have make
want promot gear groov unforn ride there go anah

more than peopl death toll itali
layin downi dont feel well
skwerl depress
ditch anoth parish some realli sick peopl thi world
jimo cant talk anymor glitch later hope thi
just gave rental discount look just couldnt turn away
order pitait nevr came they machin broke driver left what about empti belli
britney fuck amaz after just went back hote hate travel with cheapo
some peopl tricki some stop use twitter guess good get

triedand fail
look mine cant even school work sucksatleast when home that
acdc rock last night back realiti today 70000 peopl without home gt150 dead itali
well seem like troubl christian slater tri kill moment
hope some decent sleep tonight sinc didnt worth shittt last night goodnight world xoxo
want diana look great
those spot that doesnt realli show hurt like hell will doubt beauti tomorrow
heheh they were separ
babe aint callin
yeah shakili conquer ladder pointless were away receiv digit signal antenna
good morn hope everyon feel better than thi cold realli hold
tr

know monday realli when spill over tuesday
cant sleep wonder follow keep go down guess gotta more interest haha
boycot work facebook fashion
face mammoth task rest mani thing littl time
jele argh sadddddddddddddish
annoy place half hawker close want otah
twitter appear have eaten some post naughti twitter
univers hawaii hamilton librari do drink orang soda
masha alon
peopl call carpet blind light
cant believ thi want freak spoil could thi happen
watch tennant interest setfilm updat wish cardiff
struggl regard fact that danc day away other side vacat
ummm have wait until video
tummi ach much candi never learn
anyon els have troubl access their repli when click link right noth happen cant access them
feel yucki sick thi morn
same rang what progress someon els offer been reject they reckon 117118
feel terribl sick right
wake goe sleep guess should just more revis
wanna twilight again love dont have well guess suriv
babi girl sandra cantumi prayer with cantu famili strong
like know never s

your work
awww gemma hope your
where search come aoif didnt twitter must like better than
nobuuuuuuu evil evil girl
wish were play reindeer game fowler
tri write essay english pain
know havent cri that much sinc wham split
mommi school close today
nooo shudent have delet
that sound bum that miss meet anyon
tri check phone bill onlin seem like doesnt like though
manu ginobili rest season playoff
total firebug enuff enuff gonna start use safari4charl ditch debug goe mani crash
movi make think thing that dont realli wanna think about
wish there someth dinner
slept hour last night death
pick doctor note back work
just been given march order gotta some work
move move move keep those box move rawhid think gone deliri from move
chang 2pac itun
stupid spamassassin ruleqa backend discov go take some fix some point ad todo list
yeah
handyman just cancel
twiitter ammm sooooo tire right nowwww fall asleep think less lazi today
be
everyon abandon twitter
just been given march order gotta some work


quit possibl though with more slang phrase describ defiant behavior
final home rawr much work
with noth your tshirt
good moment email stop work
today wear ian hoodi tire allergi act like music over again
worst girlfriend univers
just watch digit award diana vicker could have
everybodi els hard part get someon extra effort stuf spbpuk
mighti mousei longer mighti it� get strang ohappl whyyyy
just finish work type thi internet kiosk centralworld cant wait home
veri tough isnt virtual pick peopl shake them everi penni stress
work without ninja luke
that fantast hate timesheet they forc them
have some extra pixi dust from
husband rick grimmi do freshli squeez thi week waaahhh guyzzzz
feel just woke school hope both have better rest hope that includ meet
good mornin every1 yall talk bout snow dreamt snow here disapptin when just woke have work
go lunch with feel veri sick dont want
have plant anoth potato think have overdon time will tell
feel today
use love leagu back when wigan pretti much

...

didnt homework thi weekend still doesnt care caus alreadi into
hmmm siguro naman recov yung someon that time
gonna finish watch tough love then goodnight everyon
nope clear good
complet first focu group infact difficult must admiteverybodi should

mine get sent xbox free
cours welcom
your lose
rest comfort blanket tonight
life would dull without friend
indiaari sing time
yeah that what talkin about sound better alreadi
talk amaz readi graduat alreadi hoep spend summer newyork
mayb luckier than most definit luckier than mani
watch video musiqueplu brazil cant understand what guy say doesnt matter
sometim wish that random clever word wisdom would into head
go need some tbar time soon
good comment well josi repeatedli said market fort isnt awaresg
watch some anim final pleas teacher tonight tomorrow
listen fava song decod paramor still tri find lost them oop
awwww beyond cute
go sinc have earli shop night tweeter
airway ask exodu
tomorrow will straight work think about
yeah veri random go

mine
veri cute nope function fine
about wish want wish good luck with final gonna need
have defin notic
from south seen least differ weather system time common forecast temp 6090
haha bout sabrina respond
about drive

love email that know get more follow keep comin beatin
smitti love
norri sound veri british think which guess consid just quit weird hear yourself mani thank
hell yeah
that what figur
most welcom love when friend from come over
happi girl everyth good
looooov that song beatl great music miley your great
take pic webcam
look good still think that look like hole
thing your casa mellow hope some sleep soon
twitter gone weekend sister wed beauti then danc night yesterday
dont know block tri sleep work class sweet dream
how favorit puppi thi fine sunday
enrol procedur initi good chao your section
marci playground candi 1997 who that loung chair yeah mamma thi sure dream
thi week john mayb next week
get readi lunch break
excel more time revis what elast again
beddi good relax n

thank love lot
time
gooooodmorn earli today just half hour will home from week dad
best day chat thur thi week home edit know what might work
their work cant wait more thank sweet comment
thank watch thi fight love next day
happi doubt thank
home take quick shower with scabi hug well least cute cardiac clinic
went citi with karaok sooooooooooo much then went food shop good time good time
haha how servic
great bday bday with wasnt true friend nfamili show good time with their call time msg thnx
wait call call will answer probabl talk hour
wonder go keep with three myspac facebook twitter challeng
listen music while fold laundri clean room
work from twitterberrri
friend lock himself amphad makeshft kareoke sesh audiosalv last niteit hyster
woottt south brisban haha
well veri excit what your buy that make excit yeah hardcor rock
ask
there just good night
awesom
thank your also veri sweet which
yepper have class earli tomorrow morn
joeywhi serioussmil live think morn
someon wanna thi girl 

oooh sundaysnap joey
final home such greatmemor weekend night
kate find some credibl concret sustain fashion brand love know
better news last week class befor final
stole heart
seeeein soon
glad like articl particularli good thought
follow thank alot
have sooo mani favourit movi with stelmo about last night were first one introduc rock
thi

finish everyth need some anthro project done good product night goooodnight twiggasslt3
love death
play guitar open cool stuff pour wasnt mind melt robert johnson
row
gotcha brah hahaha braddah killin
final done
haha kuroshitsujitwilight plot show drunk confus 34th through what hypocrit
road trip great bangalor coorg mangalor manip beach mysor bangalor coorg realli green beauti
made sakura green still think about someth need wednesdaysuspens
listen love
dont wanna share
like that cupcak palac sassi suggest sunshin lollipop palac
fantast see ladi today
drink more lifewat youll fine blaggh
thank everyon involv great weekend
well keep mind that steven 

that great thank share hahahaha
talk like know where fuck live 1700 walku district height bitch
love tear song
beauti song excel piano intro
ciarenn just home happi kinda nake
thank rendigitti
love hear preach world your kid realli stole show
wait green ticket releas
final sleep like hour then back write paper alot done tonight
haha make happi love bluray
felt send that hope help someon know bless goodnight
haha actual manag twice gave away other fan onli need
awesom wallpap
botak paatibayan 2009 fort 5k10k21k42k plan join
some day more question who have babi thi year
sister love unit state tara
belong with radio
good morn been hour myself who count
thank feel teenci better have goood peac
current hour ahead monday even here home time yippee have awesom monday
what
have with those mothafn snake special bobbbenet
oreo know watch jona alreadydont need repli that
backk danika regan check text messag
know noth about garageband mix musik clue nada nilch
feel like sunday aint haha
lenni meji

In [31]:
df2['clean_tweet'][0]

'upset that cant updat facebook text might result school today also blah'

In [32]:
X = df2['clean_tweet']
y = df2['label']

# train_test_split

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(max_features=5000,stop_words='english')
bow = bow.fit_transform(df2['clean_tweet'])

In [35]:
bow.dtype

dtype('int64')

In [36]:
X_train,X_test,y_train,y_test = train_test_split(bow,df2['label'],test_size=0.2,random_state = 51)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(16000, 5000)
(4000, 5000)
(16000,)
(4000,)


In [37]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train,y_train)
lr.score(X_test,y_test)

0.739

In [38]:
pred = lr.predict(X_test)
import collections
collections.Counter(pred)

Counter({1: 2155, 0: 1845})

In [39]:
from sklearn.metrics import f1_score, accuracy_score
f1_score(y_test,pred)

0.7495201535508638

In [40]:
accuracy_score(y_test,pred)

0.739

In [41]:
def cleaner(text1):
    
    text1 = clean(text1)
    text1 = text1.split()
    text2=[]
    
    for i in text1:
        if(len(i)>3):
            text2.append(i)
        
    text3=' '.join([str(ele) for ele in text2])
    return text3    

In [42]:
cleaner("i hate like 90% of my family.... i was really born into a big ass toxic bomb! mother fuckers don't give a fuck about me cuz i got money")

'hate like family really born into toxic bomb mother fuckers dont give fuck about money'

In [43]:
from flair.models import TextClassifier
from flair.data import Sentence

In [44]:
classifier = TextClassifier.load('en-sentiment')

sentence = Sentence("Okie doke!! Time for me to escape for the North while Massa's back is turned. Be on when I get home folks ")
classifier.predict(sentence)
str(sentence.labels[0])

2022-04-15 19:51:28,333 loading file C:\Users\Hp\.flair\models\sentiment-en-mix-distillbert_4.pt


'Sentence: "Okie doke ! ! Time for me to escape for the North while Massa \'s back is turned . Be on when I get home folks" → POSITIVE (0.5837)'

In [49]:
df2.head(10)

,label,id,tweet,clean_tweet
0,0,1467810672,is upset that he can't update his Facebook by ...,upset that cant updat facebook text might resu...
1,0,1467810917,@Kenichan I dived many times for the ball. Man...,dive mani time ball manag save rest bound
2,0,1467811184,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
3,0,1467811193,"@nationwideclass no, it's not behaving at all....",behav here becaus cant over there
4,0,1467811372,@Kwesidei not the whole crew,whole crew
5,0,1467811592,Need a hug,need
6,0,1467811594,@LOLTrish hey long time no see! Yes.. Rains a...,long time rain onli fine thank how
7,0,1467811795,@Tatiana_K nope they didn't have it,nope they didnt have
8,0,1467812025,@twittera que me muera ?,muera
9,0,1467812416,spring break in plain city... it's snowing,spring break plain citi snow


In [48]:

for i in range(len(df2)):
    sentence=Sentence(cleaner(df2['tweet'][i]))
    classifier.predict(sentence)
    print(str(sentence.labels[0]))


Sentence: "upset that cant update facebook texting might result school today also blah" → NEGATIVE (0.9979)
Sentence: "dived many times ball managed save rest bounds" → NEGATIVE (0.9257)
Sentence: "whole body feels itchy like fire" → NEGATIVE (0.9992)
Sentence: "behaving here because cant over there" → POSITIVE (0.5823)
Sentence: "whole crew" → POSITIVE (0.9947)
Sentence: "need" → POSITIVE (0.941)
Sentence: "long time rains only fine thanks hows" → NEGATIVE (0.8455)
Sentence: "nope they didnt have" → NEGATIVE (0.929)
Sentence: "muera" → POSITIVE (0.5629)
Sentence: "spring break plain city snowing" → POSITIVE (0.9568)
Sentence: "just repierced ears" → NEGATIVE (0.9946)
Sentence: "couldnt bear watch thought loss embarrassing" → NEGATIVE (0.999)
Sentence: "counts either never talk anymore" → NEGATIVE (0.9885)
Sentence: "wouldve been first didnt have really though snyders just doucheclown" → NEGATIVE (0.9971)
Sentence: "wish watch with miss premiere" → NEGATIVE (0.9946)
Sentence: "hollis d

IndexError: list index out of range